# Hilbert space for TNFR states

TNFR spectral states inhabit finite-dimensional Hilbert spaces that combine discrete resonant modes with continuous projections. Working in an orthonormal basis keeps expectation values and norms mechanically stable.

## Canonical structure

* Vectors live on the Hilbert sphere so coherence operators act predictably.
* Inner products use the sesquilinear form ⟨ψ, φ⟩.
* Projections return coordinates relative to any supplied orthonormal basis.

### Finite $\ell^2 \otimes L^2$ realisation
The TNFR engine realises $H_{\text{NFR}}$ as a finite section of the coupled discrete/continuous spectrum.  ``tnfr.mathematics.spaces.HilbertSpace`` provides the truncated $\ell^2$ factor with a canonical orthonormal basis and sesquilinear inner product implemented via ``numpy.vdot``.  The complementary ``tnfr.mathematics.spaces.BanachSpaceEPI`` packages the sampled $L^2$ continuum and associated coherence functional so that spectral vectors can be paired with continuous envelopes.  Together these classes make the tensor-product section explicit: ``HilbertSpace`` handles discrete projections, while ``BanachSpaceEPI`` validates and weights the continuous samples, ensuring the resulting state stays faithful to the $\ell^2 \otimes L^2$ geometry used throughout the operators.

## Smoke check: norms and expectations

The next cell computes norms and a coherence expectation value for a simple three-dimensional state. The calculation stays deterministic and exercises the mathematics helpers used throughout the engine.

In [ ]:
import numpy as np

from tnfr.mathematics.operators import CoherenceOperator
from tnfr.mathematics.runtime import coherence_expectation
from tnfr.mathematics.spaces import HilbertSpace

space = HilbertSpace(dimension=3)
psi = np.array([1.0, 0.0, 0.0], dtype=np.complex128)
phi = np.array([1.0, 1.0, 0.5], dtype=np.complex128)
phi = phi / space.norm(phi)

inner = space.inner_product(psi, phi)
norm_phi = space.norm(phi)
coherence_operator = CoherenceOperator([0.8, 0.6, 0.3])
coherence_value = coherence_expectation(phi, coherence_operator, normalise=False)

{
    "⟨ψ|φ⟩": complex(round(inner.real, 6) + 1j * round(inner.imag, 6)),
    "‖φ‖": round(norm_phi, 6),
    "coherence": round(coherence_value, 6),
    "c_min": round(coherence_operator.c_min, 6),
}


## Deterministic Hilbert-space validation
The following smoke cell checks the norm homogeneity and triangle inequality on fixed vectors and confirms that projections onto an orthonormal basis reproduce the original state, all within the ``HilbertSpace`` abstraction.

In [ ]:
import numpy as np
from tnfr.mathematics.spaces import HilbertSpace

space = HilbertSpace(dimension=3)
psi = np.array([1.0 + 0.0j, 0.5 - 0.25j, -0.5j], dtype=np.complex128)
scalar = 0.75 - 0.5j
homogeneity_lhs = space.norm(scalar * psi)
homogeneity_rhs = abs(scalar) * space.norm(psi)

u = np.array([0.8 + 0.0j, -0.2 + 0.1j, 0.3 - 0.4j], dtype=np.complex128)
v = np.array([-0.3 + 0.2j, 0.4 + 0.0j, -0.1 + 0.1j], dtype=np.complex128)
triangle_lhs = space.norm(u + v)
triangle_rhs = space.norm(u) + space.norm(v)

basis = [
    np.array([1.0, 0.0, 0.0], dtype=np.complex128),
    np.array([0.0, 1 / np.sqrt(2), 1 / np.sqrt(2)], dtype=np.complex128),
    np.array([0.0, 1 / np.sqrt(2), -1 / np.sqrt(2)], dtype=np.complex128),
]
coefficients = space.project(psi, basis)
basis_matrix = np.vstack(basis)
reconstruction = basis_matrix.conj().T @ coefficients
projection_error = float(np.max(np.abs(reconstruction - psi)))

{
    'homogeneity_diff': round(homogeneity_lhs - homogeneity_rhs, 12),
    'triangle_gap': round(triangle_lhs - triangle_rhs, 12),
    'projection_error': round(projection_error, 12),
    'coefficients': [complex(round(c.real, 6), round(c.imag, 6)) for c in coefficients],
}
